In [23]:
import requests
from bs4 import BeautifulSoup
import torch
from transformers import PreTrainedTokenizerFast
import numpy as np
import psycopg2
from pgvector.psycopg2 import register_vector
import asyncio
import aiohttp
from tqdm.asyncio import tqdm_asyncio
import nest_asyncio
nest_asyncio.apply()


In [24]:
def scrape_html(url):
    # Attempt to fetch the page's HTML content using the scraping bee API
    # response = requests.get(url)
    response = requests.get(
        url='https://app.scrapingbee.com/api/v1/',
        params={
            'api_key': 'U9048JRQFEVT5EP5HC2YX6ZHNTK3HGVOP2GGHKIGMJ73OXO6GQQFJ2MEV9HY3MPIYUPLO18WIX9WQIUD',
            'url': url,
            'wait': '3000', 
        },
        
    )
    html_content = response.text
    return html_content
insta_html = scrape_html('https://www.instagram.com')

In [25]:

async def fetch_html(session, url, api_key):
    params = {
        'api_key': api_key,
        'url': url,
        'wait': '3000',
    }
    async with session.get('https://app.scrapingbee.com/api/v1/', params=params) as response:
        return await response.text()

async def scrape_html(urls, api_key):
    connector = aiohttp.TCPConnector(limit_per_host=5)  # Max of 5 concurrent connections to the same endpoint
    async with aiohttp.ClientSession(connector=connector) as session:
        tasks = []
        for url in urls:
            task = asyncio.create_task(fetch_html(session, url, api_key))
            tasks.append(task)

        html_contents = []
        for f in tqdm_asyncio.as_completed(tasks):
            result = await f
            html_contents.append(result)
        return html_contents

# List of URLs to test
urls = ['https://www.instagram.com' for _ in range(10)]  # Example: replace with your actual URLs
api_key = 'U9048JRQFEVT5EP5HC2YX6ZHNTK3HGVOP2GGHKIGMJ73OXO6GQQFJ2MEV9HY3MPIYUPLO18WIX9WQIUD'

# Run the async scrape function and print the length of each response
async def main():
    htmls = await scrape_html(urls, api_key)
    for html in htmls:
        print(len(html))

# if __name__ == '__main__':
    # asyncio.run(main())
htmls = asyncio.run(scrape_html(urls, api_key))

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:19<00:00,  1.92s/it]


In [26]:
def html_to_text(html_content):
    # Parse the HTML content with Beautiful Soup
    soup = BeautifulSoup(html_content, 'html.parser')
    # Extract and print all text content from the HTML
    text_content = soup.get_text(separator=' ', strip=True)
    return text_content
[html_to_text(html) for html in htmls][:2]

['Instagram',
 'Instagram Meta About Blog Jobs Help API Privacy Terms Locations Instagram Lite Threads Contact Uploading & Non-Users Meta Verified English Down chevron icon Afrikaans العربية Čeština Dansk Deutsch Ελληνικά English English (UK) Español (España) Español فارسی Suomi Français עברית Bahasa Indonesia Italiano 日本語 한국어 Bahasa Melayu Norsk Nederlands Polski Português (Brasil) Português (Portugal) Русский Svenska ภาษาไทย Filipino Türkçe 中文(简体) 中文(台灣) বাংলা ગુજરાતી हिन्दी Hrvatski Magyar ಕನ್ನಡ മലയാളം मराठी नेपाली ਪੰਜਾਬੀ සිංහල Slovenčina தமிழ் తెలుగు اردو Tiếng Việt 中文(香港) Български Français (Canada) Română Српски Українська © 2024 Instagram from Meta']

In [27]:
def text_to_chunks(text_content, chunk_size):
    # Split the text content into chunks of a specific size
    chunks = [text_content[i:i+chunk_size] for i in range(0, len(text_content), chunk_size)]
    return chunks
insta_chunks = text_to_chunks(html_to_text(htmls[0]), 100)
insta_chunks

['Instagram']

In [38]:
# use the opensearch amazon model to create sparse embeddings of documents etc.
model_path = 'opensearch-neural-sparse-encoding-v1.pt'
model = torch.jit.load(model_path)
model.to('cuda')
tokenizer_file = 'tokenizer.json'
tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_file)

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [37]:
def chunks_to_sparse(descs):
    t = tokenizer(descs, return_tensors='pt', padding='max_length', truncation=True, max_length=64)
    t.to('cuda')
    with torch.no_grad():
        vecs = model(t)['output']
    np_vecs = vecs.cpu().detach().numpy()
    rfs = []
    for vec in np_vecs:
        indices = np.argwhere(vec > 0).flatten()
        tokens = tokenizer.convert_ids_to_tokens(indices)
        values = vec[indices]
        rfs.append(dict(zip(tokens, values)))
    return np_vecs, rfs
svecs = chunks_to_sparse(['oscar is a loyal farmer', 'jeremy is a hard working accountant'])
svecs[0][0].shape

(30522,)

In [9]:
# we need to persist to pg and query pg
# Connect to your postgres DB
conn = psycopg2.connect(
    host="localhost",
    port="5442",
    database="dbname",
    user="user",
    password="password"
)
# Open a cursor to perform database operations
cur = conn.cursor()
# Execute a query
cur.execute("SELECT version();")
# Retrieve query result
version = cur.fetchone()
print("Connected to:", version)
# register vector type
register_vector(conn)

Connected to: ('PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',)


In [10]:
conn.commit()
cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS test (id serial PRIMARY KEY, url text, embedding svector(30522));')
cur.executemany('INSERT INTO test (url, embedding) VALUES (%s, %s)', [('o', svecs[0][0]), ('j', svecs[0][1])])
qsvec = chunks_to_sparse('watching the computer screen')[0][0]
# N.B. <-> is euclidean which we don't want. fields returns j for euclidean and o for inner product.
# N.B. the documentation says to multiply by -1 but this is not necessary.
cur.execute('SELECT url, embedding FROM test ORDER BY (embedding <#> %s) LIMIT 1', (qsvec,))
cur.fetchone()

('j',
 '[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [11]:
## CREATE TABLES
# Table schema
# tabs: url | title | id | image bytea | status (open/bkmk/hist/web/del) | last_visited | screenshot bytea
# chunks: id | tab_id | text | embedding svector

conn.commit()
cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS tabs (id serial PRIMARY KEY, url text, title text, image bytea, status text, last_visited timestamp, screenshot bytea);')
cur.execute('CREATE TABLE IF NOT EXISTS chunks (id serial PRIMARY KEY, url text, text text, embedding svector);')
conn.commit()


In [19]:
def ingest_urls(urls):
    htmls = asyncio.run(scrape_html(urls, api_key))
    texts = [html_to_text(html) for html in htmls]
    tabs = [{'url': url, 'text': text, 'chunks': text_to_chunks(text, 100)} for url, text in zip(urls, texts)]
    chunks = [{'url': tab['url'], 'text': chunk} for tab in tabs for chunk in tab['chunks']]
    npvecs, svecs = chunks_to_sparse([chunk['text'] for chunk in chunks])
    chunks = [{**chunk, 'embedding': npvec} for chunk, npvec in zip(chunks, npvecs)]
    # start new transaction
    conn.commit()
    cur = conn.cursor()
    # insert into tabs
    cur.executemany('INSERT INTO tabs (url) VALUES (%s)', [(tab['url'],) for tab in tabs])
    # insert into chunks
    cur.executemany('INSERT INTO chunks (url, text, embedding) VALUES (%s, %s, %s)', [(chunk['url'], chunk['text'], chunk['embedding']) for chunk in chunks])
    cur.close()
    conn.commit()

urls = ['https://www.instagram.com', 'https://www.facebook.com']
ingest_urls(urls)



100%|██████████| 2/2 [00:09<00:00,  4.91s/it]


In [20]:
# show tables tabs and chunks
conn.commit()
cur = conn.cursor()
cur.execute('SELECT * FROM tabs')
cur.fetchall()

[(3, 'https://www.instagram.com', None, None, 'open', None, None),
 (5, 'https://www.instagram.com', None, None, 'open', None, None),
 (4, 'https://www.facebook.com', None, None, 'bkmk', None, None),
 (6, 'https://www.facebook.com', None, None, 'bkmk', None, None),
 (7, 'https://www.instagram.com', None, None, None, None, None),
 (8, 'https://www.facebook.com', None, None, None, None, None)]

In [21]:
def update_url_status(open_urls, bkmk_urls, hist_urls):
    cur = conn.cursor()
    cur.executemany('UPDATE tabs SET status = %s WHERE url = %s', [('open', url) for url in open_urls])
    cur.executemany('UPDATE tabs SET status = %s WHERE url = %s', [('bkmk', url) for url in bkmk_urls])
    cur.executemany('UPDATE tabs SET status = %s WHERE url = %s', [('hist', url) for url in hist_urls])
    cur.close()
    conn.commit()
update_url_status(['https://www.instagram.com'], ['https://www.facebook.com'], [])

In [22]:
query = '''
WITH ranked_chunks AS (
  SELECT tabs.*, chunks.text, (chunks.embedding <#> %s) as ip,
         ROW_NUMBER() OVER(PARTITION BY tabs.id ORDER BY (chunks.embedding <#> %s) ASC) as rank
  FROM tabs
  JOIN chunks ON tabs.url = chunks.url
)
SELECT id, url, text
FROM ranked_chunks
WHERE rank = 1
ORDER BY ip LIMIT %s;
'''
def query_similar_tabs(query_prefix, k=5):
    conn.commit()
    cur = conn.cursor()
    qsvec = chunks_to_sparse(query_prefix)[0][0]
    cur.execute(query, (qsvec, qsvec, k))
    results = cur.fetchall()
    conn.commit()
    return results

query_similar_tabs('napoleon in canada', 5)

[(3,
  'https://www.instagram.com',
  ' नेपाली ਪੰਜਾਬੀ සිංහල Slovenčina தமிழ் తెలుగు اردو Tiếng Việt 中文(香港) Български Français (Canada) Româ'),
 (5,
  'https://www.instagram.com',
  ' नेपाली ਪੰਜਾਬੀ සිංහල Slovenčina தமிழ் తెలుగు اردو Tiếng Việt 中文(香港) Български Français (Canada) Româ'),
 (7,
  'https://www.instagram.com',
  ' नेपाली ਪੰਜਾਬੀ සිංහල Slovenčina தமிழ் తెలుగు اردو Tiếng Việt 中文(香港) Български Français (Canada) Româ'),
 (4,
  'https://www.facebook.com',
  'l Français (France) 中文(简体) العربية Português (Brasil) Italiano 한국어 Deutsch हिन्दी 日本語 Sign Up Log In'),
 (6,
  'https://www.facebook.com',
  'l Français (France) 中文(简体) العربية Português (Brasil) Italiano 한국어 Deutsch हिन्दी 日本語 Sign Up Log In')]

In [16]:
from fastapi import FastAPI

app = FastAPI()


# ingest urls takes a list of urls (strings)
@app.post("/ingest_urls")
async def post_ingest_urls(urls: list[str]):
    ingest_urls(urls)
    return {"status": "success"}

# update_url_status takes a list of open, bkmk, hist urls
@app.post("/update_url_status")
async def post_update_url_status(open_urls: list[str], bkmk_urls: list[str], hist_urls: list[str]):
    update_url_status(open_urls, bkmk_urls, hist_urls)
    return {"status": "success"}

# query similar tabs takes a query prefix and returns k similar tabs
@app.get("/query_similar_tabs")
async def get_query_similar_tabs(query_prefix: str, k: int = 5):
    return query_similar_tabs(query_prefix, k)


In [ ]:
## run the server
from uvicorn import Config, Server

# Apply the patch to allow nested event loops
nest_asyncio.apply()

# Define a method to run the server
async def run_server(app, port=8000):
    config = Config(app=app, host="127.0.0.1", port=port, log_level="info")
    server = Server(config)
    await server.serve()

# Run the server in the background
import asyncio
loop = asyncio.get_event_loop()
task = loop.create_task(run_server(app))

INFO:     Started server process [9501]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:49116 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:49116 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:50044 - "GET /query_similar_tabs?query_prefix=napoleon&k=5 HTTP/1.1" 200 OK


100%|██████████| 2/2 [00:19<00:00,  9.66s/it]


INFO:     127.0.0.1:47092 - "POST /ingest_urls HTTP/1.1" 200 OK
INFO:     127.0.0.1:41176 - "POST /update_url_status HTTP/1.1" 200 OK
INFO:     127.0.0.1:41916 - "GET /query_similar_tabs?query_prefix=napoleon&k=5 HTTP/1.1" 200 OK
INFO:     127.0.0.1:57094 - "GET /query_similar_tabs?query_prefix=napoleon&k=5 HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/home/oscar/.cache/pypoetry/virtualenvs/backend-2_Www8kW-py3.10/lib/python3.10/site-packages/uvicorn/protocols/http/httptools_impl.py", line 419, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/home/oscar/.cache/pypoetry/virtualenvs/backend-2_Www8kW-py3.10/lib/python3.10/site-packages/uvicorn/middleware/proxy_headers.py", line 84, in __call__
    return await self.app(scope, receive, send)
  File "/home/oscar/.cache/pypoetry/virtualenvs/backend-2_Www8kW-py3.10/lib/python3.10/site-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/home/oscar/.cache/pypoetry/virtualenvs/backend-2_Www8kW-py3.10/lib/python3.10/site-packages/starlette/applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/home/oscar/.cache/pypoetry/virtualenvs/backend-2_Www8kW-py3.10/lib/python3.10/

INFO:     127.0.0.1:57100 - "GET /query_similar_tabs?query_prefix=napoleon&k=5 HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/home/oscar/.cache/pypoetry/virtualenvs/backend-2_Www8kW-py3.10/lib/python3.10/site-packages/uvicorn/protocols/http/httptools_impl.py", line 419, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/home/oscar/.cache/pypoetry/virtualenvs/backend-2_Www8kW-py3.10/lib/python3.10/site-packages/uvicorn/middleware/proxy_headers.py", line 84, in __call__
    return await self.app(scope, receive, send)
  File "/home/oscar/.cache/pypoetry/virtualenvs/backend-2_Www8kW-py3.10/lib/python3.10/site-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/home/oscar/.cache/pypoetry/virtualenvs/backend-2_Www8kW-py3.10/lib/python3.10/site-packages/starlette/applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/home/oscar/.cache/pypoetry/virtualenvs/backend-2_Www8kW-py3.10/lib/python3.10/

INFO:     127.0.0.1:57114 - "GET /query_similar_tabs?query_prefix=napoleon&k=5 HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/home/oscar/.cache/pypoetry/virtualenvs/backend-2_Www8kW-py3.10/lib/python3.10/site-packages/uvicorn/protocols/http/httptools_impl.py", line 419, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/home/oscar/.cache/pypoetry/virtualenvs/backend-2_Www8kW-py3.10/lib/python3.10/site-packages/uvicorn/middleware/proxy_headers.py", line 84, in __call__
    return await self.app(scope, receive, send)
  File "/home/oscar/.cache/pypoetry/virtualenvs/backend-2_Www8kW-py3.10/lib/python3.10/site-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/home/oscar/.cache/pypoetry/virtualenvs/backend-2_Www8kW-py3.10/lib/python3.10/site-packages/starlette/applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/home/oscar/.cache/pypoetry/virtualenvs/backend-2_Www8kW-py3.10/lib/python3.10/